### Some Webscraping

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_style
set_style("whitegrid")

In [124]:
import requests
from bs4 import BeautifulSoup
from time import sleep # didn't need it in the end; we could make immediate requests

In [77]:
# this initial stuff did not end up giving the data set we need so I have the correct solution below
# which I'll make clear with a markdown cell
# but it's instructive to keep this stuff anyways

url = "https://www.timeanddate.com/time/change/usa/new-york"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

In [78]:
html.status_code

200

In [79]:
print(html.text)

<!DOCTYPE html><!--
scripts and programs that download content transparent to the user are not allowed without permission
--><html lang=en><head><meta http-equiv=Content-Type content="text/html; charset=utf-8"><title>Daylight Saving Time Changes 2024 in New York, New York, USA</title><meta name=description content="Daylight Savings Time 2024 in USA – New York – New York. When does time change? Date of the next clock change and exact time to spring forward or fall back."><meta name=robots content="max-image-preview:large"><meta property="og:image" content="https://www.timeanddate.com/scripts/cityog.php?title=Time%20Changes%20in&amp;city=New%20York&amp;state=New%20York&amp;country=USA&amp;image=new-york1"><meta property="og:type" content=website><style>
@font-face{font-family:iconfont;src:url("/common/fonts/iconfont.woff2?v8") format("woff2"),url("/common/fonts/iconfont.woff?v8") format("woff"),url("/common/fonts/iconfont.ttf?v8") format("truetype"),url("/common/fonts/iconfont.svg?v8#ico

In [80]:
soup.head.title

<title>Daylight Saving Time Changes 2024 in New York, New York, USA</title>

In [81]:
select = soup.find('select') # find the feature where we can select range of years

In [82]:
select

<select class="inline" id="tb-dst-select" name="syear"><option selected="" value="2023-2027">Recent/upcoming years</option><option value="2020-2029">2020 — 2029</option><option value="2010-2019">2010 — 2019</option><option value="2000-2009">2000 — 2009</option><option value="1990-1999">1990 — 1999</option><option value="1980-1989">1980 — 1989</option><option value="1970-1979">1970 — 1979</option><option value="1960-1969">1960 — 1969</option><option value="1950-1959">1950 — 1959</option><option value="1925-1949">1925 — 1949</option><option value="1900-1924">1900 — 1924</option><option value="1850-1899">1850 — 1899</option><option value="1800-1849">1800 — 1849</option></select>

In [83]:
range_of_years = [option['value'] for option in select.find_all('option')]

In [84]:
range_of_years[::-1] #ordering the years


['1800-1849',
 '1850-1899',
 '1900-1924',
 '1925-1949',
 '1950-1959',
 '1960-1969',
 '1970-1979',
 '1980-1989',
 '1990-1999',
 '2000-2009',
 '2010-2019',
 '2020-2029',
 '2023-2027']

In [85]:
header={'Accept': '*/*'} # seems needed but not sure how

In [122]:
import re
# Sample: see if the sentence begins with 'The' and ends with 'Spain'
sentence = "The rain fell slowly in Spain"
x = re.search("^The.*Spain$", sentence)
x

<re.Match object; span=(0, 29), match='The rain fell slowly in Spain'>

For more on regular expressions, see this [link](https://www.w3schools.com/python/python_regex.asp)

In [ ]:
# the regular expressions we'll use
# Define the regex (regular expressions) pattern to match dates; the way to read is shown below
year_pattern = r'\?year=(\d{4})' 
date_pattern = r'\b\w+,\s\w+\s\d{1,2}\b'

For regex

\b: Word boundary to ensure we match whole words

\w+: Matches the day of the week (one or more word characters)

,\s: A comma followed by a space

\w+: Matches the month (one or more word characters)

\s\d{1,2}: Matches a space followed by one or two digits for the day

\b

In [87]:
# attempt by directly changing the url and making requests via GET

date_year_pairs = []

for x in range_of_years[::-1]:
    new_url="https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear="+str(x[:4])+"&eyear="+str(x[5:])+"&json=1"
    response1=requests.get(new_url,headers=header)
    print("https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear="+str(x[:4])+
        "&eyear="+str(x[5:])+"&json=1")
    dates = re.findall(date_pattern, response1.text)
    years = re.findall(year_pattern, response1.text)
    
    for i in range(len(years)):
        if i * 3 < len(dates):  # Adjust the multiplier according to the number of dates per year
            date_year_pairs.append((years[i]+', '+ dates[i * 2], years[i]+ ', '+dates[i * 2 + 1]))  # Assuming each year has two dates


https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1800&eyear=1849&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1850&eyear=1899&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1900&eyear=1924&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1925&eyear=1949&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1950&eyear=1959&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1960&eyear=1969&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1970&eyear=1979&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1980&eyear=1989&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1990&eyear=1999&json=1
https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=200

In [88]:
date_year_pairs = date_year_pairs[:-7]
date_year_pairs

[('1918, Sunday, March 31', '1918, Sunday, October 27'),
 ('1919, Sunday, March 30', '1919, Sunday, October 26'),
 ('1920, Sunday, March 28', '1920, Sunday, October 31'),
 ('1921, Sunday, April 24', '1921, Sunday, September 25'),
 ('1922, Sunday, April 30', '1922, Sunday, September 24'),
 ('1925, Sunday, April 26', '1925, Sunday, September 27'),
 ('1926, Sunday, April 25', '1926, Sunday, September 26'),
 ('1927, Sunday, April 24', '1927, Sunday, September 25'),
 ('1928, Sunday, April 29', '1928, Sunday, September 30'),
 ('1929, Sunday, April 28', '1929, Sunday, September 29'),
 ('1930, Sunday, April 27', '1930, Sunday, September 28'),
 ('1931, Sunday, April 26', '1931, Sunday, September 27'),
 ('1932, Sunday, April 24', '1932, Sunday, September 25'),
 ('1933, Sunday, April 30', '1933, Sunday, September 24'),
 ('1934, Sunday, April 29', '1934, Sunday, September 30'),
 ('1935, Sunday, April 28', '1935, Sunday, September 29'),
 ('1936, Sunday, April 26', '1936, Sunday, September 27'),
 ('

In [89]:
# there are 105 years of data but somehow we only managed to get 75 years. How?
print((2023-1918))
print(len(date_year_pairs))

105
75


It seems maybe some inconsistent data entry or something led to some goof-ups of switching the spring and fall time changes. But if you check, the dates are also not correct.

('2013, Sunday, November 4', '2013, Sunday, March 10'),

('2015, Sunday, November 3', '2015, Sunday, March 9'),

('2017, Sunday, November 2', '2017, Sunday, March 8'),
 
 Also, we're missing many years as we saw about. And 2018 has weird entries. Lastly, we're missing 2024.

### Successful solution

Let's try cheating a bit and asking for years 1800 to 2024 in one go

In [125]:
all_years_url="https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1800&eyear=2024&json=1"

In [126]:
DST= []

response2=requests.get(all_years_url,headers=header)
dates = re.findall(date_pattern, response2.text)
years = re.findall(year_pattern, response2.text)
    
for i in range(len(years)):
    if i * 3 < len(dates):  # Adjust the multiplier according to the number of dates per year
        DST.append((years[i]+', '+ dates[i * 2], years[i]+ ', '+dates[i * 2 + 1]))  # Assuming each year has two dates


In [127]:
len(DST) #somehow we only got up to 1991! 

72

In [128]:
DST # note that we're missing 1943-1944 because of WWII

[('1918, Sunday, March 31', '1918, Sunday, October 27'),
 ('1919, Sunday, March 30', '1919, Sunday, October 26'),
 ('1920, Sunday, March 28', '1920, Sunday, October 31'),
 ('1921, Sunday, April 24', '1921, Sunday, September 25'),
 ('1922, Sunday, April 30', '1922, Sunday, September 24'),
 ('1923, Sunday, April 29', '1923, Sunday, September 30'),
 ('1924, Sunday, April 27', '1924, Sunday, September 28'),
 ('1925, Sunday, April 26', '1925, Sunday, September 27'),
 ('1926, Sunday, April 25', '1926, Sunday, September 26'),
 ('1927, Sunday, April 24', '1927, Sunday, September 25'),
 ('1928, Sunday, April 29', '1928, Sunday, September 30'),
 ('1929, Sunday, April 28', '1929, Sunday, September 29'),
 ('1930, Sunday, April 27', '1930, Sunday, September 28'),
 ('1931, Sunday, April 26', '1931, Sunday, September 27'),
 ('1932, Sunday, April 24', '1932, Sunday, September 25'),
 ('1933, Sunday, April 30', '1933, Sunday, September 24'),
 ('1934, Sunday, April 29', '1934, Sunday, September 30'),
 ('

In [129]:
url_1992_2024="https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=1992&eyear=2024&json=1"

DST_1992_2024=[]

response3=requests.get(url_1992_2024,headers=header)
dates = re.findall(date_pattern, response3.text)
years = re.findall(year_pattern, response3.text)

for i in range(len(years)):
    if i * 3 < len(dates):  # Adjust the multiplier according to the number of dates per year
        DST_1992_2024.append((years[i]+', '+ dates[i * 2], years[i]+ ', '+dates[i * 2 + 1]))  # 

In [130]:
DST_1992_2024 # looks like we got only up to 2015; 
              # so our variable name is misleading but it's not my fault

[('1992, Sunday, April 5', '1992, Sunday, October 25'),
 ('1993, Sunday, April 4', '1993, Sunday, October 31'),
 ('1994, Sunday, April 3', '1994, Sunday, October 30'),
 ('1995, Sunday, April 2', '1995, Sunday, October 29'),
 ('1996, Sunday, April 7', '1996, Sunday, October 27'),
 ('1997, Sunday, April 6', '1997, Sunday, October 26'),
 ('1998, Sunday, April 5', '1998, Sunday, October 25'),
 ('1999, Sunday, April 4', '1999, Sunday, October 31'),
 ('2000, Sunday, April 2', '2000, Sunday, October 29'),
 ('2001, Sunday, April 1', '2001, Sunday, October 28'),
 ('2002, Sunday, April 7', '2002, Sunday, October 27'),
 ('2003, Sunday, April 6', '2003, Sunday, October 26'),
 ('2004, Sunday, April 4', '2004, Sunday, October 31'),
 ('2005, Sunday, April 3', '2005, Sunday, October 30'),
 ('2006, Sunday, April 2', '2006, Sunday, October 29'),
 ('2007, Sunday, March 11', '2007, Sunday, November 4'),
 ('2008, Sunday, March 9', '2008, Sunday, November 2'),
 ('2009, Sunday, March 8', '2009, Sunday, Novem

In [131]:
DST_1992_2024[0:(2015-1992+1)] 

[('1992, Sunday, April 5', '1992, Sunday, October 25'),
 ('1993, Sunday, April 4', '1993, Sunday, October 31'),
 ('1994, Sunday, April 3', '1994, Sunday, October 30'),
 ('1995, Sunday, April 2', '1995, Sunday, October 29'),
 ('1996, Sunday, April 7', '1996, Sunday, October 27'),
 ('1997, Sunday, April 6', '1997, Sunday, October 26'),
 ('1998, Sunday, April 5', '1998, Sunday, October 25'),
 ('1999, Sunday, April 4', '1999, Sunday, October 31'),
 ('2000, Sunday, April 2', '2000, Sunday, October 29'),
 ('2001, Sunday, April 1', '2001, Sunday, October 28'),
 ('2002, Sunday, April 7', '2002, Sunday, October 27'),
 ('2003, Sunday, April 6', '2003, Sunday, October 26'),
 ('2004, Sunday, April 4', '2004, Sunday, October 31'),
 ('2005, Sunday, April 3', '2005, Sunday, October 30'),
 ('2006, Sunday, April 2', '2006, Sunday, October 29'),
 ('2007, Sunday, March 11', '2007, Sunday, November 4'),
 ('2008, Sunday, March 9', '2008, Sunday, November 2'),
 ('2009, Sunday, March 8', '2009, Sunday, Novem

In [132]:
# try another time
url_2016_2024="https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=2016&eyear=2024&json=1"
DST_2016_2024=[]

response4=requests.get(url_2016_2024,headers=header)
dates = re.findall(date_pattern, response4.text)
years = re.findall(year_pattern, response4.text)

for i in range(len(years)):
    if i * 3 < len(dates):  # Adjust the multiplier according to the number of dates per year
        DST_2016_2024.append((years[i]+', '+ dates[i * 2], years[i]+ ', '+dates[i * 2 + 1])) 

In [133]:
DST_2016_2024 # wow, feels like this website really does not want to give up its info

[('2016, Sunday, March 13', '2016, Sunday, November 6'),
 ('2017, Sunday, March 12', '2017, Sunday, November 5'),
 ('2018, Sunday, March 11', '2018, Sunday, November 4'),
 ('2019, Sunday, March 10', '2019, Sunday, November 3'),
 ('2020, Sunday, March 8', '2020, Sunday, November 1'),
 ('2021, Sunday, March 14', '2021, Sunday, November 7')]

In [134]:
url_2022_2024="https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=dst&syear=2022&eyear=2024&json=1"

DST_2022_2024=[]

response5=requests.get(url_2022_2024,headers=header)
dates = re.findall(date_pattern, response5.text)
years = re.findall(year_pattern, response5.text)

for i in range(len(years)):
    if i * 3 < len(dates):  # Adjust the multiplier according to the number of dates per year
        DST_2022_2024.append((years[i]+', '+ dates[i * 2], years[i]+ ', '+dates[i * 2 + 1])) 

In [135]:
DST_2022_2024

[('2022, Sunday, March 13', '2022, Sunday, November 6'),
 ('2023, Sunday, March 12', '2023, Sunday, November 5')]

In [136]:
# I give up on 2024, I'll just add it myself
DST_2024=[('2024, Sunday, March 10', '2024, Sunday, November 3')]

In [137]:
final_DST=DST+DST_1992_2024+DST_2016_2024+DST_2022_2024+DST_2024

In [138]:
final_DST

[('1918, Sunday, March 31', '1918, Sunday, October 27'),
 ('1919, Sunday, March 30', '1919, Sunday, October 26'),
 ('1920, Sunday, March 28', '1920, Sunday, October 31'),
 ('1921, Sunday, April 24', '1921, Sunday, September 25'),
 ('1922, Sunday, April 30', '1922, Sunday, September 24'),
 ('1923, Sunday, April 29', '1923, Sunday, September 30'),
 ('1924, Sunday, April 27', '1924, Sunday, September 28'),
 ('1925, Sunday, April 26', '1925, Sunday, September 27'),
 ('1926, Sunday, April 25', '1926, Sunday, September 26'),
 ('1927, Sunday, April 24', '1927, Sunday, September 25'),
 ('1928, Sunday, April 29', '1928, Sunday, September 30'),
 ('1929, Sunday, April 28', '1929, Sunday, September 29'),
 ('1930, Sunday, April 27', '1930, Sunday, September 28'),
 ('1931, Sunday, April 26', '1931, Sunday, September 27'),
 ('1932, Sunday, April 24', '1932, Sunday, September 25'),
 ('1933, Sunday, April 30', '1933, Sunday, September 24'),
 ('1934, Sunday, April 29', '1934, Sunday, September 30'),
 ('

In [139]:
print('The number of years between 1918 and 2024 (inclusive) is:', str(2024-1918+1))
print('The numbers of years of data we have is:', str(len(final_DST)))
print('This is because 1943-44 did not observe DST due to WWII')

The number of years between 1918 and 2024 (inclusive) is: 107
The numbers of years of data we have is: 105
This is because 1943-44 did not observe DST due to WWII


In [140]:
df = pd.DataFrame(final_DST, columns=["Spring Date", "Fall Date"])

# Extract the year from the spring and fall dates
df['Year'] = df['Spring Date'].str.split(',').str[0]  # Extract the year (first part)

# Remove the year from the spring and fall dates
df['Spring Date'] = df['Spring Date'].str.replace(r'^\d{4}, ', '', regex=True)  # Remove the year
df['Fall Date'] = df['Fall Date'].str.replace(r'^\d{4}, ', '', regex=True)  # Remove the year


In [141]:
df = df[['Year', 'Spring Date', 'Fall Date']]
df

,Year,Spring Date,Fall Date
0,1918,"Sunday, March 31","Sunday, October 27"
1,1919,"Sunday, March 30","Sunday, October 26"
2,1920,"Sunday, March 28","Sunday, October 31"
3,1921,"Sunday, April 24","Sunday, September 25"
4,1922,"Sunday, April 30","Sunday, September 24"
...,...,...,...
100,2020,"Sunday, March 8","Sunday, November 1"
101,2021,"Sunday, March 14","Sunday, November 7"
102,2022,"Sunday, March 13","Sunday, November 6"
103,2023,"Sunday, March 12","Sunday, November 5"


In [142]:
# an alternative way to remove stuff using lambda functions; I'll demonstrate by removing the day

df['Spring Date'].loc[0].split(', ')

['Sunday', 'March 31']

In [143]:
df['Fall Date Without Weekday'] = df['Fall Date'].apply(lambda day: day.split(', ')[1])

In [144]:
df

,Year,Spring Date,Fall Date,Fall Date Without Weekday
0,1918,"Sunday, March 31","Sunday, October 27",October 27
1,1919,"Sunday, March 30","Sunday, October 26",October 26
2,1920,"Sunday, March 28","Sunday, October 31",October 31
3,1921,"Sunday, April 24","Sunday, September 25",September 25
4,1922,"Sunday, April 30","Sunday, September 24",September 24
...,...,...,...,...
100,2020,"Sunday, March 8","Sunday, November 1",November 1
101,2021,"Sunday, March 14","Sunday, November 7",November 7
102,2022,"Sunday, March 13","Sunday, November 6",November 6
103,2023,"Sunday, March 12","Sunday, November 5",November 5


In [145]:
# the original data frame
df=df[['Year','Spring Date', 'Fall Date']]
df

,Year,Spring Date,Fall Date
0,1918,"Sunday, March 31","Sunday, October 27"
1,1919,"Sunday, March 30","Sunday, October 26"
2,1920,"Sunday, March 28","Sunday, October 31"
3,1921,"Sunday, April 24","Sunday, September 25"
4,1922,"Sunday, April 30","Sunday, September 24"
...,...,...,...
100,2020,"Sunday, March 8","Sunday, November 1"
101,2021,"Sunday, March 14","Sunday, November 7"
102,2022,"Sunday, March 13","Sunday, November 6"
103,2023,"Sunday, March 12","Sunday, November 5"


In [146]:
df.to_csv('DST_dates.csv', index=False)